In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
data_text = \
"""ts\_1over2 & 27 & 44 & 9.81 & 154 & -5413.3549  & 20.32 m & 34.11 s &$-5413.3552\pm0.0001$&{92 [82,103]} \\  %FCIDUMP.24
rc & 32 & 50 & 13.05 & 437 &  -5412.0367 & 2.31 h  & 4.79 m &{\color{red}$-5412.0373\pm0.0001$}&{\color{red}299 [202,457]}  \\  %FCIDUMP.20
pc & 32 & 50 & 13.05 & 449 &  -5413.1870 &  4.51 h & 8.19 m & $-5413.1876\pm0.0001$ & 332 [315,349] \\  %FCIDUMP.26
2 & 33 & 50 & 14.29 & 856 & -5411.9974  &  23.93 h & 41.67 m &$-5411.9980\pm0.0001$&583 [550,619]  \\  %FCIDUMP.28
2${}^*$ & 52 & 74 & 19.85 & 530 & -5412.0364  &  1.79 d & 1.19 h &$-5412.0446\pm0.0003$&{ 6434 [6089,6803]}  \\  %FCIDUMP.29
ts\_1over2${}^*$ & 51 & 74 & 24.22 & 583 &  -5413.4598 & 2.15 d & 1.42 h &$-5413.4663\pm0.0011$&4076 [3819,4354]    \\  %FCIDUMP.25
rc${}^*$ & 51 & 74 & 24.22 & 583 & -5412.0839  & 1.82 d & 1.27 h &$-5412.0898\pm0.0008$& 4126 [3777,4516] \\  %FCIDUMP.21
pc${}^*$ & 51 & 74 & 24.22 & 481 & -5413.2293  &  1.10 d & 48.20 m &$-5413.2366\pm0.0006$& 4888 [4638,5156]  \\  %FCIDUMP.27
mo\_n2- &33&46&15.93&1558&-8694.2886&3.82 d& 2.76 h& $-8694.2897\pm0.0001$ &  1655 [1519,1805]\\  %FCIDUMP.9, Orig
% mo\_n2- &33&46&15.93&&&& & $-8694.2890\pm0.0002$ &  {\color{red}1006 [762,1344]}\\  %FCIDUMP.9, Reverse Schedule, Quadratic approach?
% mo\_n2- &33&46&15.93&1558&-8694.28954&& & $-8694.28977\pm0.00002$ &  {\color{red}560 [536,585]}\\  %FCIDUMP.9, Reverse Schedule, Fiedler, slight Quadratic approach?
mo\_n2&33&45&16.25&878&-8694.0684&1.28 d&54.88 m& $-8694.0759\pm0.0007$ &  6502 [6090,6949]  \\  %FCIDUMP.12, Orig
% mo\_n2&33&45&16.25&878&-8694.0684&1.28 d&54.88 m& $-8694.0735\pm0.0003$ &  3316 [2900,3804]  \\  %FCIDUMP.12, Reverse Schedule
% fecp2\_s0&46&58&24.49&599&-1647.2670&1.50 d&1.06 h& $\pm$ &  []  \\  %FCIDUMP.11, Orig
fecp2\_s0&46&58&24.49&544&-1647.3037&&& $-1647.3222\pm0.0022$ & {\color{red}13133 [11589,14930]}  \\  %FCIDUMP.11, Reverse Schedule, Fiedler
fecp2+\_s0.5&46&57&24.69&599&-1647.0880&1.63 d&1.15 h& $-1647.1110\pm0.0023$ & 8413 [7301,9735] \\   %FCIDUMP.10, Orig
% fecp2+\_s0.5&46&57&24.69&599&-1647.0880&1.63 d&1.15 h& $-1647.0953\pm0.0017$ & 1836 [1723,1958] \\   %FCIDUMP.10, Reverse Schedule
% fecp2+\_s0.5&46&57&24.69&495&-1647.1222&&& $-1647.1347\pm0.00258$ & 3286 [2483,4423] \\ \hline  %FCIDUMP.10, Reverse Schedule, Fiedler
4a &  24 &  39 &    8.44 &             325 &  -5103.5570 &             2.06 & &  -5103.5570 $\pm$ 0.0000 &        15 [3, 27] \\   %FCIDUMP.30
4a &  43 &  63 &   19.85 &             825 &  -5103.8866 &           684.52 &&  -5103.8879 $\pm$ 0.0001 &  1036 [972, 1100] \\   %FCIDUMP.31
pc- &  55 &  75 &   27.92 &             450 &  -5409.4560 &           292.19 &&  -5409.4660 $\pm$ 0.0033 &  2279 [544, 4014] \\   %FCIDUMP.22
I &  56 &  77 &   28.32 &             450 &  -5411.4060 &           298.16 &&  -5411.4111 $\pm$ 0.0011 &   939 [743, 1135] \\   %FCIDUMP.18
ts\_1over4a &  56 &  75 &   28.89 &             450 &  -5409.5595 &           327.62 &&  -5409.5751 $\pm$ 0.0033 & 2454 [2052, 2856] \\   %FCIDUMP.16
1\_lut\_react &  69 &  90 &   36.73 &             450 & -12049.6523 &           610.05 && -12049.6675 $\pm$ 0.0015 & 2565 [1684, 3446] \\   %FCIDUMP.15
1\_lut\_prod &  70 &  90 &   37.62 &             420 & -12049.6887 &           536.21 && -12049.6973 $\pm$ 0.0015 &  1254 [894, 1614] \\   %FCIDUMP.14
1\_lut\_ts &  70 &  90 &   37.62 &             420 & -12049.6495 &           539.38 && -12049.6574 $\pm$ 0.0008 & 1098 [1017, 1179] \\   %FCIDUMP.13
pc- &  73 &  99 &   37.77 &             400 &  -5409.5811 &           688.88 &&  -5409.5889 $\pm$ 0.0022 &  1094 [726, 1462] \\   %FCIDUMP.23
I &  75 & 101 &   39.13 &             400 &  -5411.4342 &           651.14 &&  -5411.4404 $\pm$ 0.0019 &   978 [748, 1208] \\   %FCIDUMP.19
"""

In [3]:
data_text

'ts\\_1over2 & 27 & 44 & 9.81 & 154 & -5413.3549  & 20.32 m & 34.11 s &$-5413.3552\\pm0.0001$&{92 [82,103]} \\  %FCIDUMP.24\nrc & 32 & 50 & 13.05 & 437 &  -5412.0367 & 2.31 h  & 4.79 m &{\\color{red}$-5412.0373\\pm0.0001$}&{\\color{red}299 [202,457]}  \\  %FCIDUMP.20\npc & 32 & 50 & 13.05 & 449 &  -5413.1870 &  4.51 h & 8.19 m & $-5413.1876\\pm0.0001$ & 332 [315,349] \\  %FCIDUMP.26\n2 & 33 & 50 & 14.29 & 856 & -5411.9974  &  23.93 h & 41.67 m &$-5411.9980\\pm0.0001$&583 [550,619]  \\  %FCIDUMP.28\n2${}^*$ & 52 & 74 & 19.85 & 530 & -5412.0364  &  1.79 d & 1.19 h &$-5412.0446\\pm0.0003$&{ 6434 [6089,6803]}  \\  %FCIDUMP.29\nts\\_1over2${}^*$ & 51 & 74 & 24.22 & 583 &  -5413.4598 & 2.15 d & 1.42 h &$-5413.4663\\pm0.0011$&4076 [3819,4354]    \\  %FCIDUMP.25\nrc${}^*$ & 51 & 74 & 24.22 & 583 & -5412.0839  & 1.82 d & 1.27 h &$-5412.0898\\pm0.0008$& 4126 [3777,4516] \\  %FCIDUMP.21\npc${}^*$ & 51 & 74 & 24.22 & 481 & -5413.2293  &  1.10 d & 48.20 m &$-5413.2366\\pm0.0006$& 4888 [4638,5156]  

In [4]:
data_dict_list = []
for iter, line in enumerate(data_text.split('\n')):
    print(line)
    line = line.strip()
    # print(line)
    if not line:
        continue
    if line.startswith('%'):
        continue
    fields = line.split('&')
    # Extract extrapolated energy and 95% CI with regex
    # $-5413.3552\pm0.0001$&
    energy_str = fields[8].strip().strip('{}').strip('$')
    # extrapolated_energy_match_obj = re.search(r'(-?\d+\.\d+)\\pm(\d+\.\d+)', energy_str)
    extrapolated_energy_match_obj = re.search(r'(-?\d+\.\d+)(\\pm|\s*\$\\pm\$\s*)(\d+\.\d+)', energy_str)
    extrapolated_energy = float(extrapolated_energy_match_obj.group(1))
    extrapolated_energy_energy_95CI = float(extrapolated_energy_match_obj.group(3))

    # extrapolated_energy, extrapolated_energy_energy_95CI = energy_str.split('\\pm')
    # extrapolated_energy = float(extrapolated_energy)
    # extrapolated_energy_energy_95CI = float(extrapolated_energy_energy_95CI)

    # Extract extrapolated bond dimension with regex
    # {92 [82,103]}
    bond_dimension_str = fields[9].strip()#.replace('{', '').replace('}', '').replace('\\', '')
    # print(bond_dimension_str)
    extrapolated_bond_dimension_match_obj = re.search(r"(\d+)\s*\[(\d+),\s*(\d+)\]", bond_dimension_str)
    # print(extrapolated_bond_dimension_match_obj)
    extrapolated_bond_dimension = int(extrapolated_bond_dimension_match_obj.group(1))
    extrapolated_bond_dimension_lower_bound = int(extrapolated_bond_dimension_match_obj.group(2))
    extrapolated_bond_dimension_upper_bound = int(extrapolated_bond_dimension_match_obj.group(3))

    # Extract FCIDUMP number
    fcidump_number = re.search(r'(FCIDUMP\.\d+)', fields[-1])

    data_dict = {
        'molecule_id': fields[0].strip().replace('\\', '').replace(r"${}^*$", ''),
        'n_orb': int(fields[1]),
        'n_elec': int(fields[2]),
        'log_10_hilbert_space': float(fields[3]),
        'bond_dimension': int(fields[4]),
        'dmrg_energy': float(fields[5]),
        'cpu_time': fields[6].strip(),
        'wall_time': fields[7].strip(),
        'extrapolated_energy': extrapolated_energy,
        'extrapolated_energy_95CI': extrapolated_energy_energy_95CI,
        'extrapolated_bond_dimension': extrapolated_bond_dimension,
        'extrapolated_bond_dimension_lower_bound': extrapolated_bond_dimension_lower_bound,
        'extrapolated_bond_dimension_upper_bound': extrapolated_bond_dimension_upper_bound,
        'fcidump_number': fcidump_number.group(1)

    }
    data_dict_list.append(data_dict)

ts\_1over2 & 27 & 44 & 9.81 & 154 & -5413.3549  & 20.32 m & 34.11 s &$-5413.3552\pm0.0001$&{92 [82,103]} \  %FCIDUMP.24
rc & 32 & 50 & 13.05 & 437 &  -5412.0367 & 2.31 h  & 4.79 m &{\color{red}$-5412.0373\pm0.0001$}&{\color{red}299 [202,457]}  \  %FCIDUMP.20
pc & 32 & 50 & 13.05 & 449 &  -5413.1870 &  4.51 h & 8.19 m & $-5413.1876\pm0.0001$ & 332 [315,349] \  %FCIDUMP.26
2 & 33 & 50 & 14.29 & 856 & -5411.9974  &  23.93 h & 41.67 m &$-5411.9980\pm0.0001$&583 [550,619]  \  %FCIDUMP.28
2${}^*$ & 52 & 74 & 19.85 & 530 & -5412.0364  &  1.79 d & 1.19 h &$-5412.0446\pm0.0003$&{ 6434 [6089,6803]}  \  %FCIDUMP.29
ts\_1over2${}^*$ & 51 & 74 & 24.22 & 583 &  -5413.4598 & 2.15 d & 1.42 h &$-5413.4663\pm0.0011$&4076 [3819,4354]    \  %FCIDUMP.25
rc${}^*$ & 51 & 74 & 24.22 & 583 & -5412.0839  & 1.82 d & 1.27 h &$-5412.0898\pm0.0008$& 4126 [3777,4516] \  %FCIDUMP.21
pc${}^*$ & 51 & 74 & 24.22 & 481 & -5413.2293  &  1.10 d & 48.20 m &$-5413.2366\pm0.0006$& 4888 [4638,5156]  \  %FCIDUMP.27
mo\_n2- &33&

In [5]:
data_df = pd.DataFrame(data_dict_list)
data_df["energy_variance"] = (data_df["extrapolated_energy_95CI"]/1.96)**2
data_df

,molecule_id,n_orb,n_elec,log_10_hilbert_space,bond_dimension,dmrg_energy,cpu_time,wall_time,extrapolated_energy,extrapolated_energy_95CI,extrapolated_bond_dimension,extrapolated_bond_dimension_lower_bound,extrapolated_bond_dimension_upper_bound,fcidump_number,energy_variance
0,ts_1over2,27,44,9.81,154,-5413.3549,20.32 m,34.11 s,-5413.3552,0.0001,92,82,103,FCIDUMP.24,2.603082e-09
1,rc,32,50,13.05,437,-5412.0367,2.31 h,4.79 m,-5412.0373,0.0001,299,202,457,FCIDUMP.20,2.603082e-09
2,pc,32,50,13.05,449,-5413.1870,4.51 h,8.19 m,-5413.1876,0.0001,332,315,349,FCIDUMP.26,2.603082e-09
3,2,33,50,14.29,856,-5411.9974,23.93 h,41.67 m,-5411.9980,0.0001,583,550,619,FCIDUMP.28,2.603082e-09
4,2,52,74,19.85,530,-5412.0364,1.79 d,1.19 h,-5412.0446,0.0003,6434,6089,6803,FCIDUMP.29,2.342774e-08
5,ts_1over2,51,74,24.22,583,-5413.4598,2.15 d,1.42 h,-5413.4663,0.0011,4076,3819,4354,FCIDUMP.25,3.149729e-07
6,rc,51,74,24.22,583,-5412.0839,1.82 d,1.27 h,-5412.0898,0.0008,4126,3777,4516,FCIDUMP.21,1.665973e-07
7,pc,51,74,24.22,481,-5413.2293,1.10 d,48.20 m,-5413.2366,0.0006,4888,4638,5156,FCIDUMP.27,9.371095e-08
8,mo_n2-,33,46,15.93,1558,-8694.2886,3.82 d,2.76 h,-8694.2897,0.0001,1655,1519,1805,FCIDUMP.9,2.603082e-09
9,mo_n2,33,45,16.25,878,-8694.0684,1.28 d,54.88 m,-8694.0759,0.0007,6502,6090,6949,FCIDUMP.12,1.275510e-07


In [6]:
data_df.loc[data_df["fcidump_number"] == "FCIDUMP.20", "extrapolated_energy"].values[0]

-5412.0373

In [7]:
# Get the extrapolated_energy and extrapolated_energy_95CI for
# bar_1 = ts_1over2 - rc (small) = FCIDUMP.20 - FCIDUMP.24
# bar_2 = ts_1over2 - rc (large) = FCIDUMP.21 - FCIDUMP.25
# bar_3 = ts_1over4a - I (small) = FCIDUMP.16 - FCIDUMP.18
# bar_4 = ts_1over4a - I (large) = FCIDUMP.17 - FCIDUMP.19
# bar_5 = 1_lut_ts - 1_lut_react = FCIDUMP.13 - FCIDUMP.15
# bar_6 = (mo_n2- - mo_n2) + (fecp2+_s0.5 - fecp2_s0) = FCIDUMP.9 - FCIDUMP.12 + FCIDUMP.10 - FCIDUMP.11
# Get extrapolated_energy and variance for bar_1
bar_1_extrap_energy = data_df.loc[data_df["fcidump_number"] == "FCIDUMP.20", "extrapolated_energy"].values[0] - data_df.loc[data_df["fcidump_number"] == "FCIDUMP.24", "extrapolated_energy"].values[0]
bar_1_extrap_energy_variance = data_df.loc[data_df["fcidump_number"] == "FCIDUMP.20", "energy_variance"].values[0] + data_df.loc[data_df["fcidump_number"] == "FCIDUMP.24", "energy_variance"].values[0]
bar_1_extrap_energy_95CI = 1.96*np.sqrt(bar_1_extrap_energy_variance)


# Get extrapolated_energy and variance for bar_2
bar_2_extrap_energy = data_df.loc[data_df["fcidump_number"] == "FCIDUMP.21", "extrapolated_energy"].values[0] - data_df.loc[data_df["fcidump_number"] == "FCIDUMP.25", "extrapolated_energy"].values[0]
bar_2_extrap_energy_variance = data_df.loc[data_df["fcidump_number"] == "FCIDUMP.21", "energy_variance"].values[0] + data_df.loc[data_df["fcidump_number"] == "FCIDUMP.25", "energy_variance"].values[0]
bar_2_extrap_energy_95CI = 1.96*np.sqrt(bar_2_extrap_energy_variance)

# Get extrapolated_energy and variance for bar_3
bar_3_extrap_energy = data_df.loc[data_df["fcidump_number"] == "FCIDUMP.16", "extrapolated_energy"].values[0] - data_df.loc[data_df["fcidump_number"] == "FCIDUMP.18", "extrapolated_energy"].values[0]
bar_3_extrap_energy_variance = data_df.loc[data_df["fcidump_number"] == "FCIDUMP.16", "energy_variance"].values[0] + data_df.loc[data_df["fcidump_number"] == "FCIDUMP.18", "energy_variance"].values[0]
bar_3_extrap_energy_95CI = 1.96*np.sqrt(bar_3_extrap_energy_variance)

# # Get extrapolated_energy and variance for bar_4
# bar_4_extrap_energy = data_df.loc[data_df["fcidump_number"] == "FCIDUMP.17", "extrapolated_energy"].values[0] - data_df.loc[data_df["fcidump_number"] == "FCIDUMP.19", "extrapolated_energy"].values[0]
# bar_4_extrap_energy_variance = data_df.loc[data_df["fcidump_number"] == "FCIDUMP.17", "energy_variance"].values[0] + data_df.loc[data_df["fcidump_number"] == "FCIDUMP.19", "energy_variance"].values[0]
# bar_4_extrap_energy_95CI = 1.96*np.sqrt(bar_4_extrap_energy_variance)

# Get extrapolated_energy and variance for bar_5
bar_5_extrap_energy = data_df.loc[data_df["fcidump_number"] == "FCIDUMP.13", "extrapolated_energy"].values[0] - data_df.loc[data_df["fcidump_number"] == "FCIDUMP.15", "extrapolated_energy"].values[0]
bar_5_extrap_energy_variance = data_df.loc[data_df["fcidump_number"] == "FCIDUMP.13", "energy_variance"].values[0] + data_df.loc[data_df["fcidump_number"] == "FCIDUMP.15", "energy_variance"].values[0]
bar_5_extrap_energy_95CI = 1.96*np.sqrt(bar_5_extrap_energy_variance)

# Get extrapolated_energy and variance for bar_6
bar_6_extrap_energy = data_df.loc[data_df["fcidump_number"] == "FCIDUMP.9", "extrapolated_energy"].values[0] - data_df.loc[data_df["fcidump_number"] == "FCIDUMP.12", "extrapolated_energy"].values[0] + data_df.loc[data_df["fcidump_number"] == "FCIDUMP.10", "extrapolated_energy"].values[0] - data_df.loc[data_df["fcidump_number"] == "FCIDUMP.11", "extrapolated_energy"].values[0]
bar_6_extrap_energy_variance = data_df.loc[data_df["fcidump_number"] == "FCIDUMP.9", "energy_variance"].values[0] + data_df.loc[data_df["fcidump_number"] == "FCIDUMP.12", "energy_variance"].values[0] + data_df.loc[data_df["fcidump_number"] == "FCIDUMP.10", "energy_variance"].values[0] + data_df.loc[data_df["fcidump_number"] == "FCIDUMP.11", "energy_variance"].values[0]
bar_6_extrap_energy_95CI = 1.96*np.sqrt(bar_6_extrap_energy_variance)


In [8]:
print(bar_1_extrap_energy, bar_1_extrap_energy_95CI)
print(bar_2_extrap_energy, bar_2_extrap_energy_95CI)
print(bar_3_extrap_energy, bar_3_extrap_energy_95CI)
# print(bar_4_extrap_energy, bar_4_extrap_energy_95CI)
print(bar_5_extrap_energy, bar_5_extrap_energy_95CI)
print(bar_6_extrap_energy, bar_6_extrap_energy_95CI)

1.3179000000000087 0.0001414213562373095
1.3765000000003056 0.0013601470508735444
1.83600000000024 0.003478505426185217
0.010099999999511056 0.0017000000000000001
-0.0025999999995747203 0.0032603680773802208


In [9]:
print(rf"${bar_1_extrap_energy:.3f}\\pm{bar_1_extrap_energy_95CI:.3f}$")
print(rf"${bar_2_extrap_energy:.3f}\\pm{bar_2_extrap_energy_95CI:.3f}$")
print(rf"${bar_3_extrap_energy:.3f}\\pm{bar_3_extrap_energy_95CI:.3f}$")
# print(rf"${bar_4_extrap_energy:.3f}\\pm{bar_4_extrap_energy_95CI:.3f}$")
print(rf"${bar_5_extrap_energy:.3f}\\pm{bar_5_extrap_energy_95CI:.3f}$")
print(rf"${bar_6_extrap_energy:.3f}\\pm{bar_6_extrap_energy_95CI:.3f}$")

$1.318\\pm0.000$
$1.377\\pm0.001$
$1.836\\pm0.003$
$0.010\\pm0.002$
$-0.003\\pm0.003$


In [10]:
# Same values converted to kJ/mol from Hartee
bar_1_extrap_energy_kJmol = bar_1_extrap_energy*2625.5
bar_1_extrap_energy_95CI_kJmol = bar_1_extrap_energy_95CI*2625.5
bar_2_extrap_energy_kJmol = bar_2_extrap_energy*2625.5
bar_2_extrap_energy_95CI_kJmol = bar_2_extrap_energy_95CI*2625.5
bar_3_extrap_energy_kJmol = bar_3_extrap_energy*2625.5
bar_3_extrap_energy_95CI_kJmol = bar_3_extrap_energy_95CI*2625.5
# bar_4_extrap_energy_kJmol = bar_4_extrap_energy*2625.5
# bar_4_extrap_energy_95CI_kJmol = bar_4_extrap_energy_95CI*2625.5
bar_5_extrap_energy_kJmol = bar_5_extrap_energy*2625.5
bar_5_extrap_energy_95CI_kJmol = bar_5_extrap_energy_95CI*2625.5
bar_6_extrap_energy_kJmol = bar_6_extrap_energy*2625.5
bar_6_extrap_energy_95CI_kJmol = bar_6_extrap_energy_95CI*2625.5

print(rf"${bar_1_extrap_energy_kJmol:.3f}\\pm{bar_1_extrap_energy_95CI_kJmol:.3f}$")
print(rf"${bar_2_extrap_energy_kJmol:.3f}\\pm{bar_2_extrap_energy_95CI_kJmol:.3f}$")
print(rf"${bar_3_extrap_energy_kJmol:.3f}\\pm{bar_3_extrap_energy_95CI_kJmol:.3f}$")
# print(rf"${bar_4_extrap_energy_kJmol:.3f}\\pm{bar_4_extrap_energy_95CI_kJmol:.3f}$")
print(rf"${bar_5_extrap_energy_kJmol:.3f}\\pm{bar_5_extrap_energy_95CI_kJmol:.3f}$")
print(rf"${bar_6_extrap_energy_kJmol:.3f}\\pm{bar_6_extrap_energy_95CI_kJmol:.3f}$")

$3460.146\\pm0.371$
$3614.001\\pm3.571$
$4820.418\\pm9.133$
$26.518\\pm4.463$
$-6.826\\pm8.560$
